In [79]:
import pandas as pd
import mysql.connector
import os
from dotenv import load_dotenv

In [80]:
env_path = os.path.join('../../', '.env')
load_dotenv(dotenv_path=env_path)

True

In [81]:
df = pd.read_csv('shopping_trends.csv')

In [82]:
# Establish database connection
connection = mysql.connector.connect(
    host='localhost',
    user='jjones',
    password=os.environ.get("MYSQL_PASSWORD"),
    database='customer_shopping'
)
cursor = connection.cursor(buffered=True)

In [83]:
# mapping
customers = df[[
    'Customer ID', 
    'Age', 
    'Gender', 
    'Location', 
    'Preferred Payment Method', 
    'Previous Purchases',
    'Frequency of Purchases'
]]
# Insert customers
for _, row in customers.iterrows():
    cursor.execute(
        """
        INSERT INTO customer (customerID, age, gender, location, preferred_payment, historical_purchases, purchase_freq)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """,
        (row['Customer ID'], row['Age'], row['Gender'], row['Location'], row['Preferred Payment Method'], row['Previous Purchases'], row['Frequency of Purchases'])
    )
    
connection.commit()

In [84]:
items = df[[
    # 'Item ID'
    'Item Purchased',
    'Category',
    'Purchase Amount (USD)',
    # 'Tax Rate'
    # 'Description',
    'Size',
    'Color',
    # 'Avg Rating'
]]

items = items.drop_duplicates(subset=['Item Purchased', 'Category', 'Size', 'Color'])

# Insert items
for _, row in items.iterrows():
    cursor.execute(
        """
        INSERT INTO item (name, category, price, tax_rate, description, size, color, avg_rating)
        VALUES (%s, %s, %s, 0.01, NULL, %s, %s, NULL)
        """,
        (row['Item Purchased'], row['Category'], row['Purchase Amount (USD)'], row['Size'], row['Color'])
    )
    
connection.commit()

In [85]:
subscriptions = df[[
    'Customer ID',
    'Subscription Status'
    # start date
    # end date
]]

subscriptions = subscriptions.drop_duplicates(subset=['Customer ID'])
subscriptions = subscriptions[subscriptions['Subscription Status'] == 'Active']

# Insert subscriptions
for _, row in subscriptions.iterrows():
    cursor.execute(
        """
        INSERT INTO subscription (customerID, start_date, end_date)
        VALUES (%s, NOW(), NULL)
        """,
        (row['Customer ID'])
    )
    
connection.commit()

In [86]:
reviews = df[[
    'Customer ID',
    # key for item
    'Item Purchased',
    'Category',
    'Purchase Amount (USD)',
    'Size',
    'Color',
    'Review Rating'
]]

reviews = reviews.drop_duplicates(subset=['Customer ID', 'Item Purchased', 'Category', 'Size', 'Color'])

# Insert reviews
for _, row in reviews.iterrows():
    cursor.execute(
        """
        SELECT itemID FROM item
        WHERE name = %s AND color = %s AND size = %s AND category = %s
        """,
        (row['Item Purchased'], row['Color'], row['Size'], row['Category'])
    )
    item_id = cursor.fetchone()[0]
    
    if item_id:  # Ensure item_id is not None or empty
        cursor.execute(
            """
            INSERT INTO review (customerID, itemID, rating, review_text)
            VALUES (%s, %s, %s, NULL)
            """,
            (row['Customer ID'], item_id, row.get('Review Rating', None))  # Use .get() to handle missing 'Review Rating'
        )
    else:
        print(f"Item not found for: {row['Item Purchased']}, {row['Color']}, {row['Size']}, {row['Category']}")
connection.commit()


In [87]:
purchases = df[[
    # purchase id
    'Item Purchased',
    'Customer ID',
    # 'Date',
    'Payment Method',
    'Shipping Type',
    'Season',
    # 'Subtotal',
    'Purchase Amount (USD)',
    'Category',
    'Size',
    'Color'
]]

# Insert purchases
for _, row in purchases.iterrows():
    
    cursor.execute(
        """
        INSERT INTO purchase (customerID, date, payment_method, shipping_type, season, subtotal, total)
        VALUES (%s, NOW(), %s, %s, %s, NULL, %s)
        """,
        (row['Customer ID'], row['Payment Method'], row['Shipping Type'], row['Season'], row['Purchase Amount (USD)'])
    )
    
    cursor.execute(
        """
        SELECT LAST_INSERT_ID()
        """
    )
    
    purchase_id = cursor.fetchone()[0]
    if purchase_id is None:
        continue
        
    cursor.execute(
        """
        SELECT itemID FROM item
        WHERE name = %s AND color = %s AND size = %s AND category = %s
        """,
        (row['Item Purchased'], row['Color'], row['Size'], row['Category'])
    )
    item_id = cursor.fetchone()[0]
    
    if item_id:  # Ensure item_id is not None or empty
        cursor.execute(
            """
            INSERT INTO cart (purchaseID, itemID)
            VALUES (%s, %s)
            """,
            (purchase_id, item_id)
        )
    else:
        print(f"Item not found for: {row['Item Purchased']}, {row['Color']}, {row['Size']}, {row['Category']}")
    
connection.commit()



In [88]:
price_reductions = df[[
    'Item Purchased',
    'Category',
    'Size',
    'Color',
    'Customer ID',
    'Discount Applied',
    'Promo Code Used'
]]

price_reductions = price_reductions[(price_reductions['Discount Applied'] == 'Yes') | (price_reductions['Promo Code Used'] == 'Yes')]
price_reductions = price_reductions.drop_duplicates(subset=['Item Purchased', 'Customer ID'])

# # Insert price reductions
for _, row in price_reductions.iterrows():
    promo_type = 'discount' if row['Discount Applied'] == 'Yes' else 'promo'
    
    cursor.execute(
        """
        INSERT INTO price_reduction (reduction_type, promo_type, reduction_amount, code, description)
        VALUES ('rate', %s, 0.1, NULL, NULL)
        """,
        (promo_type,)
    )
    
    cursor.execute(
        """
        SELECT LAST_INSERT_ID()
        """
    )
    
    reduction_id = cursor.fetchone()[0]
    
    if reduction_id is None:
        continue
    
    cursor.execute(
        """
        SELECT p.purchaseID 
        FROM purchase p 
        JOIN cart c
            ON p.purchaseID = c.purchaseID
        JOIN item i
            ON c.itemID = i.itemID
        WHERE i.name = %s and p.customerID = %s and i.color = %s and i.size = %s and i.category = %s
        """,
        (row['Item Purchased'], row['Customer ID'], row['Color'], row['Size'], row['Category'])
    )
    purchase_id = cursor.fetchone()[0]
    
    if purchase_id:
        cursor.execute(
            """
            INSERT INTO purchase_price_reduction (purchaseID, price_reductionID)
            VALUES (%s, %s)
            """,
            (purchase_id, reduction_id)
        )
    else:
        print(f"Purchase not found for: {row['Item Purchased']}, {row['Customer ID']}, {row['Color']}, {row['Size']}, {row['Category']}")
connection.commit()


In [89]:
# Commit changes and close connection
connection.commit()
cursor.close()
connection.close()